In [ ]:
import msprime
import pickle
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate as integrate
%matplotlib inline

# Functions for running trait simulations:

In [ ]:
def full_pop_reps(Ne_sim, mu, pi, rr=0.0):
    genotypes = []
    c_rate = lambda t: np.exp(rr*2.*Ne_sim*t)
    T2_2 = ETj(2, 2, c_rate)[0] # get the expected pairwise coal time in units of 2Ne
    LL = round(pi / (4.*mu*Ne_sim*T2_2)) # calculate the necessary number of sites
    replicates = msprime.simulate(
        mutation_rate=mu,
        num_replicates=LL,
        population_configurations = [msprime.PopulationConfiguration(growth_rate=rr,
                                                                     sample_size=2*Ne_sim,
                                                                     initial_size=Ne_sim)])
    for j, tree_sequence in enumerate(replicates):
        if tree_sequence.get_num_mutations() > 0:
            variant = next(tree_sequence.variants())
            genotypes.append(variant.genotypes)
        if tree_sequence.get_num_mutations() > 1:
            print("Warning: more than one mutation")
    return(np.array(genotypes, dtype=np.int32))

def nt_div(genotypes):
    result = 0
    psum = 0
    for ii in range(genotypes.shape[1]):
        for jj in range(ii+1, genotypes.shape[1]):
            result += np.sum(np.abs(genotypes[:,ii] - genotypes[:,jj]))
            psum += 1
    return result/psum

def phen_pop(Ne_sim, mu, pi, mut_dist, rr=0.0):
    genotypes = np.transpose(full_pop_reps(Ne_sim, mu, pi, rr=rr))
    while len(genotypes.shape) == 1: # only take pops where at least one mutation occurs
        genotypes = np.transpose(full_pop_reps(Ne_sim, mu, pi, rr=rr))
    return(genotypes * mut_dist(genotypes.shape[1]))

def pop_kurt(phenotypes):
    phens = np.sum(phenotypes, axis=1)
    mean_phen = np.mean(phens)
    m4 = np.mean((phens - mean_phen)**4)
    m2 = np.var(phens)
    return(m4 / m2**2)

def pop_var(phenotypes):
    phens = np.sum(phenotypes, axis=1)
    return(np.var(phens))

def pop_m4(phenotypes):
    phens = np.sum(phenotypes, axis=1)
    mean_phen = np.mean(phens)
    m4 = np.mean((phens - mean_phen)**4)
    return(m4)

def var_sq(phenotypes):
    phens = np.sum(phenotypes, axis=1)
    return(np.var(phens)**2)

# Functions for computing first-order approximation

In [ ]:
def rho(j, t, eta):
    return np.exp(-scipy.special.binom(int(j), 2)*integrate.quad(eta, 0, t, limit=100)[0])

def comb_term(j, k, n):
    assert j >= k, "invalid integers"
    return ((-1)**(j-k)*(2*j - 1)*np.prod(list(range(k, k + j -1)))*
            np.prod(list(range(n, n - j, -1)))/
            (math.factorial(k)*math.factorial(j - k)*
             np.prod(list(range(n, n + j)))))

def P_Ant(k, t, n, eta):
    result = 0
    for jj in range(k, n + 1):
        result += (rho(jj, t, eta)*
                   comb_term(jj, k , n))
    return result

def ETj(k, n, eta):
    PP = lambda t: P_Ant(k, t, n, eta)
    return integrate.quad(PP, 0, scipy.inf, limit=100)

# Run simulations

Define pairwise difference parameters and the number of reps for different simulations.

In [ ]:
pi_set = [0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 10.0]
nreps = [500, 500, 500, 500, 400, 300, 200, 100]
test_Ne = 1000

Running the simulations...
Uncomment these lines to rerun the simulations. Otherwise load simulation results that were used to create figures in manuscript.

In [ ]:
#sim_phens_norm = []
#for ii, pi in enumerate(pi_set):
#    print(pi)
#    sim_phens_norm.append([])
#    for jj in range(nreps[ii]):
#        sim_phens_norm[ii].append(phen_pop(test_Ne, 1e-6, pi, mut_dist=lambda n: np.random.normal(size=n), rr=0.0))

In [ ]:
#sim_phens_lap = []
#for ii, pi in enumerate(pi_set):
#    print(pi)
#    sim_phens_lap.append([])
#    for jj in range(nreps[ii]):
#        sim_phens_lap[ii].append(phen_pop(test_Ne, 1e-6, pi, mut_dist=lambda n: np.random.laplace(size=n), rr=0.0))

In [ ]:
#sim_phens_norm_growth = []
#for ii, pi in enumerate(pi_set):
#    print(pi)
#    sim_phens_norm_growth.append([])
#    for jj in range(nreps[ii]):
#        sim_phens_norm_growth[ii].append(phen_pop(test_Ne, 1e-6, pi, 
#                                                  mut_dist=lambda n: np.random.normal(size=n), rr=1.0/test_Ne))

In [ ]:
#sim_phens_lap_growth = []
#for ii, pi in enumerate(pi_set):
#    print(pi)
#    sim_phens_lap_growth.append([])
#    for jj in range(nreps[ii]):
#        sim_phens_lap_growth[ii].append(phen_pop(test_Ne, 1e-6, pi, 
#                                                  mut_dist=lambda n: np.random.laplace(size=n), rr=1.0/test_Ne))

Load in simulations that have already been run.

In [ ]:
with open("sim_phens_norm.pyc", "rb") as fin:
     sim_phens_norm = pickle.load(fin)

In [ ]:
with open("sim_phens_lap.pyc", "rb") as fin:
    sim_phens_lap = pickle.load(fin)

In [ ]:
with open("sim_phens_norm_growth.pyc", "rb") as fin:
    sim_phens_growth_norm = pickle.load(fin)

In [ ]:
with open("sim_phens_lap_growth.pyc", "rb") as fin:
    sim_phens_growth_lap = pickle.load(fin)

Calculate the kurtosis in each simulated population.

In [ ]:
pop_kurts_norm = []
for ii, pi in enumerate(pi_set):
    pop_kurts_norm.append([0]*nreps[ii])
    for jj in range(nreps[ii]):
        pop_kurts_norm[ii][jj] = pop_kurt(sim_phens_norm[ii][jj])

In [ ]:
pop_kurts_lap = []
for ii, pi in enumerate(pi_set):
    pop_kurts_lap.append([0]*nreps[ii])
    for jj in range(nreps[ii]):
        pop_kurts_lap[ii][jj] = pop_kurt(sim_phens_lap[ii][jj])

In [ ]:
pop_kurts_growth_lap = []
for ii, pi in enumerate(pi_set):
    pop_kurts_growth_lap.append([0]*nreps[ii])
    for jj in range(nreps[ii]):
        pop_kurts_growth_lap[ii][jj] = pop_kurt(sim_phens_growth_lap[ii][jj])

In [ ]:
pop_kurts_growth_norm = []
for ii, pi in enumerate(pi_set):
    pop_kurts_growth_norm.append([0]*nreps[ii])
    for jj in range(nreps[ii]):
        pop_kurts_growth_norm[ii][jj] = pop_kurt(sim_phens_growth_norm[ii][jj])

Calculate expected kurtosis under first-order approximation

In [ ]:
ET4_4 = ETj(4, 4, lambda t: np.exp(t))[0]
ET3_4 = ETj(3, 4, lambda t: np.exp(t))[0]
ET2_4 = ETj(2, 4, lambda t: np.exp(t))[0]
ET2_2 = ETj(2, 2, lambda t: np.exp(t))[0]
AA = (ET4_4 - (1./6.)*ET3_4 - (1./9.)*ET2_4)/ET2_2
BB = ((1./6.)*ET3_4 + (1./9.)*ET2_4)/ET2_2

Now plot everything.

In [ ]:
plt.figure(1, figsize=(10, 10))
plt.subplot(221)
plt.boxplot(pop_kurts_norm, positions=pi_set, showmeans=True);
plt.plot(pi_set, [3]*len(pi_set), "--r", label="first-order kurtosis approximation")
plt.tick_params(labelsize=7)
plt.yscale("log", basey=3)
plt.xlabel("expected pairwise differences")
plt.ylabel("kurtosis")
plt.legend()
plt.title("Normal mutations - constant size")

plt.subplot(222)
plt.boxplot(pop_kurts_lap, positions=pi_set, showmeans=True);
plt.plot(pi_set, [3]*len(pi_set), "--r", label="first-order kurtosis approximation")
plt.tick_params(labelsize=7)
plt.yscale("log", basey=3)
plt.xlabel("expected pairwise differences")
plt.ylabel("kurtosis")
plt.legend()
plt.title("Laplace mutations - constant size")

plt.subplot(224)
plt.boxplot(pop_kurts_growth_lap, positions=pi_set, showmeans=True);
plt.plot(pi_set, 3 + 6.*AA/(0.5*np.array(pi_set) + 6.*BB), "--r", label="first-order kurtosis approximation")
plt.tick_params(labelsize=7)
plt.yscale("log", basey=3)
plt.xlabel("expected pairwise differences")
plt.ylabel("kurtosis")
plt.legend()
plt.title("Laplace mutations - exponential growth")

plt.subplot(223)
plt.boxplot(pop_kurts_growth_norm, positions=pi_set, showmeans=True);
plt.plot(pi_set, 3 + 3.*AA/(0.5*np.array(pi_set) + 3.*BB), "--r", label="first-order kurtosis approximation")
plt.tick_params(labelsize=7)
plt.yscale("log", basey=3)
plt.xlabel("expected pairwise differences")
plt.ylabel("kurtosis")
plt.legend()
plt.title("Normal mutations - exponential growth")